<a href="https://colab.research.google.com/github/riffarhan/Contextual-vs-Non-Contextual-Word-Embedding-on-Detecting-Multiclass-Depressive-Tweets-using-LSTM/blob/main/Word2vec%20%2B%20LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Section 1

##IMPORT

In [ ]:
!pip3 install ftfy
import tensorflow as tf
import keras
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")
import ftfy
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from math import exp
from numpy import sign

from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from tensorflow.keras.optimizers import Adam
from nltk import PorterStemmer
from keras.initializers import Constant

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, GlobalAveragePooling
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.8 MB/s 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##CONSTANTS

In [ ]:
# Reproducibility
np.random.seed(1234)

DEPRES_NROWS = 3200  # number of rows to read from DEPRESSIVE_TWEETS_CSV
RANDOM_NROWS = 12000 # number of rows to read from RANDOM_TWEETS_CSV
MAX_SEQUENCE_LENGTH = 140 # Max tweet size
MAX_NB_WORDS = 9000
EMBEDDING_DIM = 300
TRAIN_SPLIT = 0.6
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS= 10

LOAD DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#FILE PATHS
DEPRESSIVE_TWEETS_CSV = '/content/drive/MyDrive/Colab Notebooks/DATA SOURCE 1/dataset/depressive_tweets_processed.csv'
RANDOM_TWEETS_CSV = '/content/drive/MyDrive/Colab Notebooks/DATA SOURCE 1/dataset/Sentiment Analysis Dataset.csv'
EMBEDDING_FILE = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/archive 2/DatasetBaru.csv')

In [ ]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
df['Sentiment'].value_counts()

#Section 2

##Preprocessing

Preprocessing the tweets in order to:



*   Remove links and images
*   Remove hashtags
*   Remove @ mentions
*Remove emojis
*Remove stop words
*Remove punctuation
*Get rid of stuff like "what's" and making it "what is'
*Stem words so they are all the same tense (e.g. ran -> run)


In [ ]:
# Expand Contraction
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [ ]:
def clean_tweets(tweets, sentiments):
    cleaned_tweets = []
    cleaned_sentiments = [] 
    for i, tweet in enumerate(tweets):
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            #remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.>)|(pic\.twitter\.com\/.)", " ", tweet).split())
            
            #fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)
            
            #expand contraction
            tweet = expandContractions(tweet)

            #remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            #stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            #stemming words
            tweet = PorterStemmer().stem(tweet)
            
            cleaned_tweets.append(tweet)
            cleaned_sentiments.append(sentiments[i])

    return cleaned_tweets, cleaned_sentiments

In [ ]:
# merubah type data dalam record menjadi format string
df.SentimentText=df.SentimentText.astype(str)
#@title
# merubah type data dalam record sentimentText menjadi format string
#depressive_tweets_df=depressive_tweets_df.astype(str)

In [ ]:
#depressive_tweets_arr = [x for x in depressive_tweets_df[5]]
random_tweets_arr = [x for x in df['SentimentText']]
#X_d = clean_tweets(depressive_tweets_arr)
X_r, y_r = clean_tweets(random_tweets_arr, df['Sentiment'])

##Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_r)
sequences_r = tokenizer.texts_to_sequences(X_r)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

##Padding

In [ ]:
data_r = pad_sequences(sequences_r, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data_r tensor:', data_r.shape)

##Word Embedding

In [ ]:
EMBEDDING_FILE = '/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
nb_words = min(MAX_NB_WORDS, len(word_index))

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for (word, idx) in word_index.items():
    if word in word2vec.vocab and idx < MAX_NB_WORDS:
        embedding_matrix[idx] = word2vec.word_vec(word)

In [ ]:
a = np.zeros((nb_words, EMBEDDING_DIM))
print(a)
print(EMBEDDING_DIM)
print(nb_words)

In [ ]:
nb_words

##Data Splitting & One Hot Encoding

In [ ]:
# defining feature matrix(X) and response vector(y)
X = data_r
y = y_r
y = np.array(y)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = pd.get_dummies(y)
y = np.array(y)

In [ ]:
# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=1)

##Model Architecture

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf
import keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(embedding_matrix), 
                               output_dim=EMBEDDING_DIM, 
                               input_length=MAX_SEQUENCE_LENGTH, 
                               embeddings_initializer=Constant(embedding_matrix)),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(300, return_sequences = True),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(150, activation = 'relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['acc'])
print(model.summary())

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Checkpoint Farhan/"
callbacks_list = [ModelCheckpoint(checkpoint_path+"New_Word2vec + Regularization",
                                  monitor='val_acc',
                                  save_best_only=True,
                                  mode='max',
                                  verbose=1)]

In [ ]:
print(tf.keras.utils.plot_model)

In [ ]:
early_stop = EarlyStopping(monitor='val_acc', patience=5) #Stop training when a monitored metric has stopped improving.

hist = model.fit(X_train, y_train, \
        validation_data=(X_test, y_test), \
        epochs=10, batch_size=16, shuffle=True, 
        callbacks = early_stop)

#Section 3

##Model Accuracy & Loss

In [ ]:
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

##Confusion Matrix

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
report_dict = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), digits=3, output_dict = True)
pd.DataFrame(report_dict)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import pandas as pd
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
plt.figure(3)
cf_matrix = ConfusionMatrixDisplay.from_predictions(y_test.argmax(axis=1), y_pred.argmax(axis=1), cmap='Blues')
plt.title('Confusion Matrix')
plt.show()